## Querying databases
We've already seen a couple of SQL queries when creating a database table and checking if data was present. Now we will go beyond those basics queries to do slightly more advanced queries, like searching and filtering.

In [13]:
# Work with an in-memory SQLite database again
import sqlite3
connection = sqlite3.connect(':memory:')

In [14]:
# Create a table again for holding a path and size, just like before
table = 'CREATE TABLE files (id integer primary key, path TEXT, bytes INTEGER)'
cursor = connection.cursor()
cursor.execute(table)
connection.commit()

There is a _large_files.py_ file that has a `files` variable which holds a list of tuples with some sample data we can use to populate the database. Import that module and use the list to iterate over it and then populate the database

In this section you will use a special SQL syntax in SQLite to insert values from Python into the SQL query.

In [17]:
from large_files import files

for metadata in files:
    query = 'INSERT INTO files(path, bytes) VALUES(?, ?)'
    # the execute() method accepts a query and optionally a tuple with values 
    # corresponding to the question marks in VALUES
    cursor.execute(query, metadata)
    connection.commit()


So far we've seen `CREATE` and `INSERT`. Let's try a new SQL statement to make a selection. The `SELECT` statement produces a result from one or more tables and from one or many rows. 

Note the particular (and strict) order of SQL statements:

- `SELECT`
- `FROM`
- `WHERE`
- `GROUP BY`
- `HAVING`
- `ORDER BY`

Since SQLite returns an iterator as a result always, then it is required to loop over the resulting object. Create a query to count the items in the `files` table. This query will use the `COUNT()` function that produces a number:

In [22]:
query = 'SELECT COUNT(id) from files'

for i in cursor.execute(query):
    print(i)

(2001,)


## Extracting distinct row data
Counting items is a good way of checking the amount of items that exist in the database. Without using `COUNT(id)` the previous query would've produced two thousand entries. Use the `LIMIT` statement to set the maximum number of entries that can be produced, then remove the `COUNT(id)` function and use `*` instead to use all rows

In [23]:
query = 'SELECT * from files LIMIT(10)'
for i in cursor.execute(query):
    print(i)

(1, '/Users/alfredo/Library/Application Support/.DS_Store', 6148)
(2, '/Users/alfredo/Library/Application Support/.settings', 557)
(3, '/Users/alfredo/Library/Application Support/Firefox/installs.ini', 71)
(4, '/Users/alfredo/Library/Application Support/Firefox/profiles.ini', 286)
(5, '/Users/alfredo/Library/Application Support/Firefox/Profiles/3zgfj4k7.default/times.json', 47)
(6, '/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/compatibility.ini', 220)
(7, '/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/favicons.sqlite-wal', 524704)
(8, '/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/addons.json', 24)
(9, '/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/.parentlock', 0)
(10, '/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/logins.json', 643)


Using `*` means every row in the table. The table in this case is `files`. The next query specifies using the `id` row only. 

**Exercise:** Update the cell contents so that it shows paths instead.

In [24]:
query = 'SELECT id FROM files LIMIT(10)'
for i in cursor.execute(query):
    print(i)

(1,)
(2,)


The next query uses ID and Bytes. Update the query once again to select two rows in the table: `bytes` and `path`.

In [31]:
query = 'SELECT id,bytes FROM files LIMIT(10)'
for i in cursor.execute(query):
    print(i)

(1, 6148)
(2, 557)
(3, 71)
(4, 286)
(5, 47)
(6, 220)
(7, 524704)
(8, 24)
(9, 0)
(10, 643)


## Extracting distinct data
You now know how to extract data from certain rows and how to limit that data. Next, we'll use more SQL statements to further find and filter out results so that you can get specific results.

**Exercise:** Use the next query to find 10 files that are bigger than 1mb (1000000 bytes) using a new statement (`WHERE`):

In [33]:
query = 'SELECT path FROM files WHERE bytes>1000000 LIMIT(10)'
for i in cursor.execute(query):
    print(i)

('/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/places.sqlite-wal', 2131512)
('/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/favicons.sqlite', 5242880)
('/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/places.sqlite', 5242880)
('/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/security_state/data.safe.bin', 4902142)
('/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/storage/permanent/chrome/idb/3870112724rsegmnoittet-es.sqlite', 14385152)
('/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/gmp-gmpopenh264/1.8.1.1/libgmpopenh264.dylib', 1012096)
('/Users/alfredo/Library/Application Support/Firefox/Profiles/vxy45f54.default-release/gmp-widevinecdm/4.10.2209.1/libwidevinecdm.dylib', 9698128)
('/Users/alfredo/Library/Application Support/Spotify/User Data/WidevineCDM/4.10.

The query shows the paths but not the sizes. 

**Exercise:** Try updating the previous query to show both the path and the size. 

In [ ]:
query = ''
for i in cursor.execute(query):
    print(i)

SQL has many helper functions, in this case the next query uses `MAX()` which can find the highest value in a column. Do you think that `LIMIT(10)` makes sense in this query? Why? What happens if you remove the `LIMIT(10)` clause?

**Exercise:** Remote the `LIMIT()` clause and check your results

In [38]:
query = 'SELECT path,MAX(bytes) FROM files LIMIT(10)'
for i in cursor.execute(query):
    print(i)

('/Users/alfredo/Library/Application Support/Quake3/baseq3/pak0.pk3', 479493658)


SQL queries can be compounded for more conditionals. In Python, you can make the query more readable by using triple quotes and adding the queries in a multi-line variable.

**Exercise:** Use other conditions to match different sizes and limit to a different number of entries returned

In [61]:
query = """
SELECT path,bytes FROM files 
    WHERE bytes>3000000 
    AND bytes<4400000 LIMIT(100)
"""
for i in cursor.execute(query):
    print(i)

('/Users/alfredo/Library/Application Support/Code/CachedData/e7d7e9a9348e6a8cc8c03f877d39cb72e5dfb1ff/extension-ca2c1aa0053e7287d1f9d28629439dd1.code', 3226912)
('/Users/alfredo/Library/Application Support/Code/Service Worker/CacheStorage/46d6e50d2ffc17893d2014fea87951879db7be6e/c39febc2-ae3b-41f4-a090-89f74e7869d1/848e0c80e6507d26_0', 3952816)
('/Users/alfredo/Library/Application Support/Code/Service Worker/CacheStorage/8bb0178b48a96ad4ef4a0b4a835ad58c89c7d663/4c006a21-b197-439d-b3c6-6a2bb7ef9a68/848e0c80e6507d26_0', 3952816)
('/Users/alfredo/Library/Application Support/Code/Service Worker/CacheStorage/e7d86aa06a3a0bb38c95db406df77476663db84b/ee1d8a80-45aa-4e2d-a19e-36095014c44d/848e0c80e6507d26_0', 3952816)
('/Users/alfredo/Library/Application Support/Code/Service Worker/CacheStorage/c857019fd12bbf4dd5269cd1b1f74afd717e2616/867b6dd5-c514-4bfb-92fc-30ad367b606d/848e0c80e6507d26_0', 3952816)
('/Users/alfredo/Library/Application Support/Code/Service Worker/CacheStorage/4ae4a6eedbab83c9b

## Searching
Sometimes you can't tell exactly what is it that you are looking for in a query. SQL allows for matching patterns. In the file paths situation, you might know that a specific file ends with `.zip` but you don't know where it is. 

**Exercise:** Use the `LIKE` operator to match and find a cache file related to an Address Book application.

In [65]:
query = """
SELECT path,bytes FROM files 
    WHERE path LIKE '%AddressBook%'
"""
for i in cursor.execute(query):
    print(i)

('/Users/alfredo/Library/Application Support/AddressBook/AddressBook-v22.abcddb-wal', 3790432)
('/Users/alfredo/Library/Application Support/AddressBook/Sources/CCB8CED3-EBC7-43D3-AB10-6FA77C01BB88/AddressBook-v22.abcddb', 2818048)
('/Users/alfredo/Library/Application Support/AddressBook/Sources/CCB8CED3-EBC7-43D3-AB10-6FA77C01BB88/AddressBook-v22.abcddb-wal', 3333112)


Using `%` means to match any text of zero or more characters. So `%AddressBook%` is very lenient for anything before it and after it. Try adding a condition that filters the result by size. Anything over 2MB (or 2000000 bytes) and see if you can reduce the amount of output.

There are other variations for search like using an underscore (`_`). That means any single character. If you know that a file prefix or suffix is, you could use this to fine-tune your search.

**Exercise:** Use other search items for the paths found in your filesystem and try to match them